# Libraries Importations

In [2]:
#We import the librarires

from selenium import webdriver
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
import time
import csv
import json
from selenium.webdriver.common.by import By

# We define some basic and useful functions

In [3]:
def clickElement(category, classe, information):
    """This function allows you to click on an element of the HTML page"""
    element = driver.find_element(By.XPATH, "//" + category + "[@" + classe + "='" + information + "']")
    element.click()


def fillField(category, class_name, information, text):
    """This function allows you to fill in a field of the HTML page"""
    elem = driver.find_element(By.XPATH, "//" + category + "[@" + class_name + "='" + information + "']")
    elem.click()
    elem.clear()
    elem.send_keys(text)


def openLink(link):
    """This function allows you to open a link"""
    driver.get(link)
    time.sleep(6)
    

# This function will get all the href links that have the a field data-test = "job-link"
def getAll_JobDescriptionLinks():
    """This function allows you to get all the href links that have the a field data-test = "job-link" """
    # We get all the links of the page
    links = driver.find_elements(By.XPATH, "//a[@data-test='job-link']")
    # We get the href of all the links
    hrefs = [link.get_attribute("href") for link in links]
    return hrefs

def getAll_JobDescriptionLinks_Indeed():
    """This function allows you to get all the href links that have the a field data-test = "job-link" """
    # We get all the links of the page
    links = driver.find_elements(By.XPATH, "//a[@class='jcs-JobTitle css-jspxzf eu4oa1w0']")
    # We get the href of all the links
    hrefs = [link.get_attribute("href") for link in links]
    return hrefs


def assembleLinkGlassdoor(base, job_name, from_age):
    """This function allows you to assemble the link of the page to be scraped"""
    # Example: https://www.glassdoor.com/Job/machine-learning-jobs-SRCH_KO0,16.htm?fromAge=1
    job_name = job_name.replace(" ", "-")
    lenght_job_name = len(job_name)
    link = base + "Job/" + job_name + "-jobs-SRCH_KO0," + str(lenght_job_name) + ".htm?fromAge=" + str(from_age)
    return link


def assembleLinkIndeed(base, job_name, from_age, location):
    """This function allows you to assemble the link of the page to be scraped"""
    # Example: https://www.indeed.com/jobs?q=machine+learning+engineer&l=United+States&fromage=1
    job_name = job_name.replace(" ", "+")
    location = location.replace(" ", "+")
    link = base + "jobs?q=" + job_name + "&l=" + location + "&fromage=" + str(from_age)
    return link


# Glassdoor Webscrapping

In [20]:
# We set the link of the page we want to scrape
base = "https://www.glassdoor.com/"
job_names = ["machine learning engineer", "data scientist", "AI engineer"]
from_age = 1


def getJobOffer_Glassdoor(base, job_name, from_age):
    """Get all the job offers for a specific search"""
    # We create the link
    link = assembleLinkGlassdoor(base, job_name, from_age)
    # We open the link
    openLink(link)

    # We get all the job offers for this search
    job_offers = []
    previous_page_offers = []
    current_page_offers = []
    while True:
        # We get all the job offers
        current_page_offers = getAll_JobDescriptionLinks()
        # We check if the current page offers are the same as the previous page offers
        if current_page_offers == previous_page_offers:
            break
        else:
            # We add the current page offers to the job offers
            job_offers += current_page_offers
            # We update the previous page offers
            previous_page_offers = current_page_offers
            # We click on the next button
            try:
                clickElement("span", "alt", "Close")
                time.sleep(1)
            except:
                pass
            try:
                clickElement("button", "id", "onetrust-accept-btn-handler")
                time.sleep(0.5)
            except:
                pass
            try:
                clickElement("button", "class", "e1jbctw80 ei0fd8p1 css-1n14mz9 e1q8sty40")
                time.sleep(1)
            except:
                pass
            clickElement("button", "aria-label", "Next")
            time.sleep(2)
    return job_offe


def main_Glassdoor(base, job_names, from_age):
    """Get all the job offers for a list of job names"""
    # We initialize the driver through Firefox
    global driver
    driver = webdriver.Firefox()
    # We get all the job offers for each job name
    job_offers = []
    for job_name in job_names:
        job_offers.append(getJobOffer_Glassdoor(base, job_name, from_age))
    # We close the driver
    driver.close()
    return job_offers

def saveData_Glassdoor(job_offers, job_names, file_path, field_names):
    for job_offers, job_name in zip(job_offers, job_names):
        with open(file_path, "a", newline="") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=field_names)
            # If the file is empty, we write the header
            if csvfile.tell() == 0:
                writer.writeheader()
            for job_offer in job_offers:
                writer.writerow(
                    {
                        "link": job_offer,
                        "job_name_searched": job_name,
                        "date": time.strftime("%Y-%m-%d"),
                        "platform": "glassdoor",
                    }
                )

In [5]:
job_offers = main_Glassdoor(base, job_names, from_age)

print("Glassdoor Web Scraping is done.")

file_path = "../data/job_offers_glassdoor.csv"
field_names = ["link", "job_name_searched", "date", "platform"]

#saveData_Glassdoor(job_offers, job_names, file_path, field_names)

#print("Saving the Glassdoor data in a csv file is done.")

Glassdoor Web Scraping is done.


In [17]:
def getJobDescription_Title_Glassdoor(job_offers):
    """Get the job description for a list of job offers"""
    # We initialize the driver through Firefox
    global driver
    driver = webdriver.Firefox()
    # We get the job description for each job offer
    job_descriptions = []
    job_titles = []
    for job_category in job_offers:
        descriptions = []
        titles = []
        for job_offer in job_category:
            # We open the link
            try:
                openLink(job_offer)
                time.sleep(2)
                # We get the job description
                description = driver.find_element(By.ID, "JobDescriptionContainer").text
                # We get the job title
                title = driver.find_element_by_xpath(
                    "//div[@data-test='job-title']"
                ).text
                # We add the job description  and title to their respective lists
                descriptions.append(description)
                titles.append(title)
            except:
                descriptions.append("None")
                titles.append("None")
                print("Unable to get information for job offer: " + job_offer)
        # We add the job descriptions and titles to their respective lists
        job_descriptions.append(descriptions)
        job_titles.append(titles)
    # We close the driver
    driver.close()
    return job_descriptions, job_titles

job_descriptions, job_titles = getJobDescription_Title_Glassdoor(job_offers)


C:\Users\CYTech Student\AppData\Local\Temp\ipykernel_17960\2819886039.py:20: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  title = driver.find_element_by_xpath(


# Indeed WebScrapping

In [21]:
# We set the link of the page we want to scrape
base = "https://www.indeed.com/"
job_names = ["machine learning engineer", "data scientist", "AI engineer"]
from_age = 1
location = "United States"

def getJobOffer_Indeed(base, job_name, from_age, location):
    """Get all the job offers for a specific search"""
    # We create the link
    link = assembleLinkIndeed(base, job_name, from_age, location)
    # We open the link
    openLink(link)
    time.sleep(2)

    # We get all the job offers for this search
    job_offers = []

    while True:
        # We get all the job offers of the first page
        current_page_offers = getAll_JobDescriptionLinks_Indeed()
        # We add the current page offers to the job offers
        job_offers += current_page_offers

        # We try to click on the next button
        try:
            clickElement("a", "data-testid", "pagination-page-next")
            time.sleep(2)
        except:
            break

    return job_offers


def main_Indeed(base, job_names, from_age, location):
    """Get all the job offers for a list of job names"""
    # We initialize the driver through Firefox
    global driver
    driver = uc.Chrome(use_subprocess=True)
    driver.maximize_window() 
    # We get all the job offers for each job name
    job_offers = []
    for job_name in job_names:
        job_offers.append(getJobOffer_Indeed(base, job_name, from_age, location))
    # We close the driver
    driver.close()
    return job_offers

def saveData_Indeed(job_offers, job_names, file_path, field_names):
    for job_offers, job_name in zip(job_offers, job_names):
        with open(file_path, "a", newline="") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=field_names)
            # If the file is empty, we write the header
            if csvfile.tell() == 0:
                writer.writeheader()
            for job_offer in job_offers:
                writer.writerow(
                    {
                        "link": job_offer,
                        "job_name_searched": job_name,
                        "date": time.strftime("%Y-%m-%d"),
                        "platform": "indeed",
                    }
                )


In [22]:
job_offers = main_Indeed(base, job_names, from_age, location)

print("Indeed Web Scraping is done.")

file_path = "../data/job_offers_indeed.csv"
field_names = ["link", "job_name_searched", "date", "platform"]

saveData_Indeed(job_offers, job_names, file_path, field_names)

print("Saving the Indeed data in a csv file is done.")

Indeed Web Scraping is done.
Saving the Indeed data in a csv file is done.


# Filtering Jobs Using ChatGPT

In [18]:
chat_gpt_link = "https://chat.openai.com/"
credentials_file_path = "../data/credentials.json"

def get_credentials(file_path):
    """This function allows you to get the credentials from a json file"""
    with open(file_path) as json_file:
        credentials = json.load(json_file)
    
    email = credentials["email"]
    password = credentials["password"]

    return email, password

def connect(link, email, password):
    """This function allows you to connect to the chat GPT using your Google account"""
    # We open the link
    openLink("https://chat.openai.com/")
    # We click on login
    clickElement("button", "class", "btn relative btn-primary")
    time.sleep(2)
    # We click on continue with Google
    clickElement("button", "data-provider", "google")
    time.sleep(2)
    # We fill in the email field
    fillField("input", "type", "email", email)
    time.sleep(1)
    # We click on next
    clickElement("button", "class", "VfPpkd-LgbsSe VfPpkd-LgbsSe-OWXEXe-k8QpJ VfPpkd-LgbsSe-OWXEXe-dgl2Hf nCP5yc AjY5Oe DuMIQc LQeN7 qIypjc TrZEUc lw1w4b")
    time.sleep(2)
    # We fill in the password field
    fillField("input", "type", "password", password)
    time.sleep(1)
    # We click on next
    clickElement("button", "class", "VfPpkd-LgbsSe VfPpkd-LgbsSe-OWXEXe-k8QpJ VfPpkd-LgbsSe-OWXEXe-dgl2Hf nCP5yc AjY5Oe DuMIQc LQeN7 qIypjc TrZEUc lw1w4b")
    time.sleep(5)
    # We close the pop-up
    clickElement("button", "class", "btn relative btn-neutral ml-auto")
    time.sleep(1)
    # We close the second pop-up
    clickElement("button", "class", "btn relative btn-neutral ml-auto")
    time.sleep(1)
    # We close the third pop-up
    clickElement("button", "class", "btn relative btn-primary ml-auto")
    time.sleep(1)

def openNewChat(text):
    # We click on the button to open a new chat
    clickElement("a", "class", "flex py-3 px-3 items-center gap-3 transition-colors duration-200 text-white cursor-pointer text-sm rounded-md border border-white/20 hover:bg-gray-500/10 mb-1 flex-shrink-0")
    time.sleep(1)
    # We fill in the chat field
    fillField("textarea", "id", "prompt-textarea", text)
    time.sleep(3)
    # We click on the send button
    clickElement("button", "class", "absolute p-1 rounded-md bottom-[10px] md:bottom-3 md:p-2 md:right-3 dark:hover:bg-gray-900 dark:disabled:hover:bg-transparent right-2 disabled:text-gray-400 enabled:bg-brand-purple text-white transition-colors disabled:opacity-40")
    time.sleep(3)
    # We get the response
    for i in range(3):
        try:
            response = driver.find_element_by_xpath("//div[@class='markdown prose w-full break-words dark:prose-invert light']")
            response = response.text
            break
        except:
            time.sleep(3)
            pass
    return response

def continueChat(text):
    # We fill in the chat field
    fillField("textarea", "id", "prompt-textarea", text)
    time.sleep(1)
    # We click on the send button
    clickElement("button", "class", "absolute p-1 rounded-md bottom-[10px] md:bottom-3 md:p-2 md:right-3 dark:hover:bg-gray-900 dark:disabled:hover:bg-transparent right-2 disabled:text-gray-400 enabled:bg-brand-purple text-white transition-colors disabled:opacity-40")
    time.sleep(3)
    # We get all the responses
    for i in range(3):
        try:
            responses = driver.find_elements_by_xpath("//div[@class='markdown prose w-full break-words dark:prose-invert light']")
            break
        except:
            time.sleep(3)
            pass
    # We take only the last response
    response = responses[-1].text
    return response


def main_ChatGPT(chat_gpt_link, credentials_file_path, context_text, question_text, job_descriptions, job_titles):
    # We initialize the driver through Firefox
    global driver
    driver = uc.Chrome(use_subprocess=True)
    driver.maximize_window() 

    # We get the credentials
    email, password = get_credentials(credentials_file_path)

    # We connect to the chat GPT
    connect(chat_gpt_link, email, password)
    count = 0
    analysis = []
    for job_category in job_descriptions:
        analysis_category = []
        for job_description in job_category:
            # We open a new chat
            text = context_text
            response = openNewChat(text)
            # We continue the chat
            text = question_text + job_description
            response = continueChat(text)
            analysis_category.append(response)
            # We increment the count
            count += 1
            # Print each 50 job descriptions
            if count % 50 == 0:
                print("We have analyzed " + str(count) + " job descriptions")
        analysis.append(analysis_category)
    
    # We close the driver
    driver.close()
    return analysis
    

context_text = """You are an assistant that will analyze Job Description as Input and answer if it fits the requirements of the users (Location, experience, etc).
Each message will have this format:
"Category Requirement 1: Details
Category Requirement 2: Details
[etc]
Job Description"
And you will answer in a JSON format by 1 (yes) or 0 (no) for each Category Requirement name
If you see a Job Description named "None", it means that the Job Description is not available so you have to answer 0 (no) for each Category Requirement name"""

question_text = """
Relevant Field 1: Is the Job Description related to MACHINE LEARNING ENGINEER Job (1 (yes) or 0 (no))
Relevant Field 2: Is the Job Description related to ARTIFICIAL INTELLIGENCE Job (1 (yes) or 0 (no))
Relevant Field 3: Is the Job Description related to DATA SCIENTIST Job (1 (yes) or 0 (no))
Relevant Field 4: Is the Job Description related to DATA ENGINEER Job (1 (yes) or 0 (no))
Relevant Field 5: Is the Job Description related to SOFTWARE ENGINEER Job (1 (yes) or 0 (no))
Experience: Is the minimum experience required is equal or greater than 4 years (1 (yes) or 0 (no))
US Nationality: Is the Job Description for US Citizens only (1 (yes) or 0 (no))
Job Description:
"""
    
main_ChatGPT(chat_gpt_link, credentials_file_path, context_text, question_text, job_descriptions, job_titles)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[@class='flex py-3 px-3 items-center gap-3 transition-colors duration-200 text-white cursor-pointer text-sm rounded-md border border-white/20 hover:bg-gray-500/10 mb-1 flex-shrink-0']"}
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00B38893+48451]
	(No symbol) [0x00ACB8A1]
	(No symbol) [0x009D5058]
	(No symbol) [0x00A00467]
	(No symbol) [0x00A0069B]
	(No symbol) [0x00A2DD92]
	(No symbol) [0x00A1A304]
	(No symbol) [0x00A2C482]
	(No symbol) [0x00A1A0B6]
	(No symbol) [0x009F7E08]
	(No symbol) [0x009F8F2D]
	GetHandleVerifier [0x00D98E3A+2540266]
	GetHandleVerifier [0x00DD8959+2801161]
	GetHandleVerifier [0x00DD295C+2776588]
	GetHandleVerifier [0x00BC2280+612144]
	(No symbol) [0x00AD4F6C]
	(No symbol) [0x00AD11D8]
	(No symbol) [0x00AD12BB]
	(No symbol) [0x00AC4857]
	BaseThreadInitThunk [0x75B600C9+25]
	RtlGetAppContainerNamedObjectPath [0x77687B